# Configuración Inicial
Este bloque configura las rutas, variables y estructuras necesarias para procesar los datos.


In [21]:
import os
import pandas as pd
import re

# Configuración general
YEAR = 2020
RAW_FOLDER = "../data/raw"
PROCESSED_FOLDER = "../data/processed"
FILE_PATH_XLSX = f"{RAW_FOLDER}/data_{YEAR}.xlsx"
FILE_PATH_XLSB = f"{RAW_FOLDER}/data_{YEAR}.xlsb"
OUTPUT_PATH = f"{PROCESSED_FOLDER}/{YEAR}/pluviometros-eda-{YEAR}.csv"

# Mapeo de columnas
COLUMN_MAPPING = {
    'Fecha': ['Fecha', 'Date', 'Día', 'Dia'],
    'Obrador Kiteni': ['Base Operativa Kiteni', 'Kiteni', 'Obrador Kiteni'],
    'PS#1': ['PS1', 'PS#1', 'PS-1'],
    'KP 007+400': ['KP 007+400', 'KP 007+400', 'KP 7+400'],
    'KP 018+000': ['KP-18'],
    'KP 030+000': ['KP-30'],
    'KP 040+000': ['KP-40'],
    'KP 043+600': ['KP-43+600'],
    'KP 046+100': ['KP-46+100'],
    'KP 050+000': ['KP-50'],
    'KP 057+000': ['KP-57'],
    'KP 064+000': ['KP-64'],
    'KP 071+000': ['KP-71'],
    'KP 082+000': ['KP-82'],
    'KP 090+000': ['KP-90'],
    'KP 095+000': ['KP-95'],
    'PS#2': ['PS2', 'PS#2','PS-2', 'Planta de Selección #2'],
    'KP-126': ['KP-126'],
    'KP 127+000': ['KP-126'],
    'KP 142+000': ['KP-126'],
    'KP-181+900': ['KP-181+900'],
    'KP 184+039': ['KP-184','KP-184+039'],
    'PS#3': ['PS3', 'PS#3',  'PS-3', 'Planta de Selección #3'],
    'PS#4/XV10007': ['PS-4'],
}


# Funciones Auxiliares
Este bloque contiene las funciones necesarias para procesar los datos:
1. Detección de encabezados.
2. Identificación de columnas relevantes.
3. Manejo de valores faltantes (`NaNs`).
4. Agregar información sobre rupturas.
5. Identificar outliers.


In [22]:
def detect_header_row(df, max_rows=15):
    """Detecta la fila que contiene los encabezados reales."""
    for row in range(max_rows):
        possible_header = df.iloc[row].values
        if any(isinstance(val, str) and len(val.strip()) > 0 for val in possible_header):
            return row
    return 0  # Si no se detecta, usar la primera fila

def detect_relevant_columns(df, column_mapping):
    """Detecta columnas relevantes basadas en nombres similares."""
    detected_columns = {}
    for expected_col, possible_names in column_mapping.items():
        for col in df.columns:
            if any(re.search(rf"\b{re.escape(name)}\b", str(col), re.IGNORECASE) for name in possible_names):
                detected_columns[expected_col] = col
                break
    return detected_columns

def handle_missing_values(df, method='global_mean', monthly_averages=None):
    """Reemplaza NaNs con el promedio por columna (global o mensual)."""
    for col in df.columns:
        if col != 'Fecha':
            if method == 'global_mean':
                # Promedio global por columna
                mean_value = df[col].replace(0, pd.NA).mean(skipna=True)
                df[col] = df[col].fillna(mean_value)
            elif method == 'monthly_mean' and monthly_averages is not None:
                # Promedio mensual basado en datos disponibles
                df['Mes'] = pd.to_datetime(df['Fecha']).dt.month
                df[col] = df.apply(
                    lambda row: monthly_averages.loc[row['Mes'], col]
                    if pd.isnull(row[col]) else row[col], axis=1
                )
    return df

def add_rupture_column(df, rupture_dates):
    """Agrega una columna con información sobre rupturas basada en fechas específicas."""
    df['Ruptura'] = df['Fecha'].apply(lambda x: 1 if x in rupture_dates else 0)
    return df

def detect_outliers(df):
    """
    Detecta outliers (valores extremos) en las columnas numéricas de un DataFrame
    usando el rango intercuartílico (IQR).
    """
    outlier_info = {}  # Diccionario para guardar los resultados
    for col in df.columns:
        if col not in ['Fecha', 'Ruptura', 'Mes']:
            Q1 = df[col].quantile(0.25)  # Primer cuartil (25%)
            Q3 = df[col].quantile(0.75)  # Tercer cuartil (75%)
            IQR = Q3 - Q1  # Rango intercuartílico

            # Límites inferior y superior
            lower_bound = Q1 - 1.5 * IQR
            upper_bound = Q3 + 1.5 * IQR

            # Filtrar outliers
            outliers = df[(df[col] < lower_bound) | (df[col] > upper_bound)]

            # Guardar los resultados
            outlier_info[col] = {
                'lower_bound': lower_bound,
                'upper_bound': upper_bound,
                'outliers': outliers
            }

            print(f"Columna: {col}")
            print(f" - Límite inferior: {lower_bound}")
            print(f" - Límite superior: {upper_bound}")
            print(f" - Total de outliers: {len(outliers)}\n")

    return outlier_info



# Carga de Datos
Este bloque carga los datos desde el archivo Excel y consolida todas las hojas.


In [23]:
if os.path.exists(FILE_PATH_XLSX):
    file_path, engine = FILE_PATH_XLSX, None
elif os.path.exists(FILE_PATH_XLSB):
    file_path, engine = FILE_PATH_XLSB, 'pyxlsb'
else:
    raise FileNotFoundError(f"No se encontró archivo para el año {YEAR}")

excel_data = pd.ExcelFile(file_path, engine=engine)
sheet_names = excel_data.sheet_names
print(f"Hojas disponibles: {sheet_names}")

Hojas disponibles: ['ENERO', 'FEBRERO', 'MARZO', 'ABRIL', 'MAYO', 'JUNIO', 'JULIO', 'AGOSTO', 'SEPTIEMBRE', 'OCTUBRE', 'NOVIEMBRE', 'DICIEMBRE']


In [24]:
all_processed_data = pd.DataFrame()

if os.path.exists(FILE_PATH_XLSX):
    file_path, engine = FILE_PATH_XLSX, None
elif os.path.exists(FILE_PATH_XLSB):
    file_path, engine = FILE_PATH_XLSB, 'pyxlsb'
else:
    raise FileNotFoundError(f"No se encontró archivo para el año {YEAR}")

excel_data = pd.ExcelFile(file_path, engine=engine)
sheet_names = excel_data.sheet_names
print(f"Hojas disponibles: {sheet_names}")


Hojas disponibles: ['ENERO', 'FEBRERO', 'MARZO', 'ABRIL', 'MAYO', 'JUNIO', 'JULIO', 'AGOSTO', 'SEPTIEMBRE', 'OCTUBRE', 'NOVIEMBRE', 'DICIEMBRE']


In [25]:
for sheet_name in sheet_names:
    print(f"\nProcesando hoja: {sheet_name}")

    try:
        # Detección de encabezados
        sheet_preview = excel_data.parse(sheet_name, nrows=15)
        header_row = detect_header_row(sheet_preview)
        #print(f"Encabezado detectado en la fila: {header_row}")

        # Carga completa de datos
        sheet_data = excel_data.parse(sheet_name, skiprows=header_row)
        #print(f"Columnas disponibles antes del mapeo: {sheet_data.columns.tolist()}")
        #display(sheet_data.head())  # Ver datos iniciales

        # Eliminar columnas 'Unnamed'
        sheet_data = sheet_data.loc[:, ~sheet_data.columns.str.contains('^Unnamed')]
        #print(f"Columnas después de eliminar 'Unnamed': {sheet_data.columns.tolist()}")
        #display(sheet_data.head())  # Ver datos después de eliminar 'Unnamed'

        # Renombrar columnas según el mapeo directo
        new_column_names = {}

        # Identificar columnas que coinciden con el mapeo
        for expected_col, possible_names in COLUMN_MAPPING.items():
            for col in sheet_data.columns:
                if any(re.search(rf"\b{re.escape(name)}\b", str(col), re.IGNORECASE) for name in possible_names):
                    new_column_names[col] = expected_col

        # Renombrar columnas según el mapeo
        sheet_data = sheet_data.rename(columns=new_column_names)

        # Opcional: Eliminar columnas que no están en el mapeo
        # Descomenta las siguientes líneas si deseas eliminar las columnas no mapeadas
        # mapped_columns = list(new_column_names.keys())
        # sheet_data = sheet_data.loc[:, mapped_columns]

        #print(f"Columnas después del mapeo: {sheet_data.columns.tolist()}")
        #display(sheet_data.head())


        # Generar columna Fecha
        if 'Fecha' not in sheet_data.columns and 'Dia' in sheet_data.columns:
            sheet_data['Fecha'] = sheet_data['Dia'].apply(
                lambda x: f"{YEAR}-{sheet_name}-{int(x):02d}" if pd.notnull(x) and isinstance(x, (int, float)) else None
            )
            sheet_data.drop(columns=['Dia'], inplace=True)

        sheet_data['Fecha'] = pd.to_datetime(sheet_data['Fecha'], errors='coerce')
        #print(f"Datos después de generar la columna 'Fecha':")
        #display(sheet_data.head())  # Ver datos después de generar 'Fecha'

        sheet_data['Mes'] = sheet_data['Fecha'].dt.to_period('M')
        # Sustituir NaN por la media de cada mes
        numeric_columns = sheet_data.select_dtypes(include=['float64', 'int64']).columns
        for col in numeric_columns:
            sheet_data[col] = sheet_data.groupby('Mes')[col].transform(lambda x: x.fillna(x.mean()))

        # Concatenar datos procesados
        all_processed_data = pd.concat([all_processed_data, sheet_data], ignore_index=True)
        #Borramos la columna KP-244 porque tiene NaN en los primeros siete meses del año
        all_processed_data = all_processed_data.drop(columns=['KP-244'])
        #Borramos la columna KP-199+800 porque tiene NaN de octubre a diciembre y no aparece como columna en los siguientes años
        all_processed_data = all_processed_data.drop(columns=['KP-199+800'])

    except Exception as e:
        print(f"Error procesando la hoja {sheet_name}: {e}")

print("\nDatos consolidados:")
display(all_processed_data.head())


Procesando hoja: ENERO
Error procesando la hoja ENERO: "['KP-244'] not found in axis"

Procesando hoja: FEBRERO
Error procesando la hoja FEBRERO: "['KP-244'] not found in axis"

Procesando hoja: MARZO
Error procesando la hoja MARZO: "['KP-244'] not found in axis"

Procesando hoja: ABRIL
Error procesando la hoja ABRIL: "['KP-244'] not found in axis"

Procesando hoja: MAYO
Error procesando la hoja MAYO: "['KP-244'] not found in axis"

Procesando hoja: JUNIO
Error procesando la hoja JUNIO: "['KP-244'] not found in axis"

Procesando hoja: JULIO
Error procesando la hoja JULIO: "['KP-244'] not found in axis"

Procesando hoja: AGOSTO

Procesando hoja: SEPTIEMBRE

Procesando hoja: OCTUBRE
Error procesando la hoja OCTUBRE: "['KP-199+800'] not found in axis"

Procesando hoja: NOVIEMBRE
Error procesando la hoja NOVIEMBRE: "['KP-199+800'] not found in axis"

Procesando hoja: DICIEMBRE
Error procesando la hoja DICIEMBRE: "['KP-199+800'] not found in axis"

Datos consolidados:


,Fecha,Obrador Kiteni,PS#1,KP-7+400,KP 018+000,KP 030+000,KP 040+000,KP 043+600,KP 046+100,KP 050+000,...,KP 082+000,KP 090+000,KP 095+000,PS#2,KP 142+000,KP-181+900,KP 184+039,PS#3,PS#4/XV10007,Mes
0,2020-01-01,5.0,73.2,98.0,74.1,82.4,20.3,0.0,25.2,29.0,...,24.0,20.1,4.4,9.2,6.2,0.0,0.0,1.0,2.0,2020-01
1,2020-01-02,41.4,9.2,12.0,29.2,26.6,33.4,0.0,80.8,55.8,...,5.0,19.3,39.6,11.8,15.0,0.0,0.0,0.2,0.6,2020-01
2,2020-01-03,2.8,32.0,71.0,35.3,59.8,18.6,0.0,28.6,40.4,...,5.0,8.6,4.6,10.6,19.1,0.0,0.0,0.9,0.8,2020-01
3,2020-01-04,0.0,0.0,0.0,0.0,33.8,43.4,0.0,38.2,54.0,...,0.0,0.0,1.2,0.0,3.0,0.0,0.0,0.4,0.2,2020-01
4,2020-01-05,13.6,4.4,17.0,19.4,15.2,4.2,0.0,4.2,0.7,...,6.5,8.4,14.0,24.2,10.0,0.0,0.0,0.2,0.4,2020-01


In [26]:
# Mostrar resumen general de NaN por columna
# Mostrar cantidad total de NaN por columna
print("Cantidad total de valores NaN por columna:")
total_nan = all_processed_data.isnull().sum()
print(total_nan)

# Identificar columnas con valores NaN y su tipo de dato
print("\nColumnas con valores NaN y sus tipos:")
nan_columns = total_nan[total_nan > 0].index  # Columnas con al menos un NaN
for col in nan_columns:
    print(f"Columna: {col}, Tipo: {all_processed_data[col].dtype}, Total NaN: {total_nan[col]}")

# Analizar NaN por mes y por columna
# Excluir la columna de agrupamiento ('Mes') y calcular NaN por mes y por columna
print("\nCantidad de NaN por mes y por columna:")
nan_analysis = (
    all_processed_data.drop(columns=['Mes'])  # Excluye la columna 'Mes' si existe
    .groupby(all_processed_data['Mes'])       # Agrupa usando la columna 'Mes'
    .apply(lambda x: x.isnull().sum())        # Calcula NaN por columna
)
print(nan_analysis)



Cantidad total de valores NaN por columna:
Fecha             0
Obrador Kiteni    0
PS#1              0
KP-7+400          0
KP 018+000        0
KP 030+000        0
KP 040+000        0
KP 043+600        0
KP 046+100        0
KP 050+000        0
KP 057+000        0
KP 064+000        0
KP 071+000        0
KP 082+000        0
KP 090+000        0
KP 095+000        0
PS#2              0
KP 142+000        0
KP-181+900        0
KP 184+039        0
PS#3              0
PS#4/XV10007      0
Mes               0
dtype: int64

Columnas con valores NaN y sus tipos:

Cantidad de NaN por mes y por columna:
         Fecha  Obrador Kiteni  PS#1  KP-7+400  KP 018+000  KP 030+000  \
Mes                                                                      
2020-01      0               0     0         0           0           0   
2020-02      0               0     0         0           0           0   
2020-03      0               0     0         0           0           0   
2020-04      0               0     

# Limpieza de Datos y Análisis
Este bloque agrega información de rupturas, detecta outliers y calcula promedios mensuales.


In [27]:
# Fechas confirmadas de rupturas
rupture_dates = [
    '2004-12-22',  # Primera Falla
    '2005-09-16',  # Segunda Falla
    '2005-11-24',  # Tercera Falla
    '2006-03-04',  # Cuarta Falla
    '2015-04-30',  # Quinta Falla
    '2016-01-19',  # Sexta Falla
    '2018-02-03',  # Séptima Falla
]

# Crear columna Ruptura
all_processed_data['Ruptura'] = all_processed_data['Fecha'].apply(lambda x: 1 if pd.to_datetime(x) in pd.to_datetime(rupture_dates) else 0)

# Validar si se agregó correctamente
print(all_processed_data[['Fecha', 'Ruptura']].drop_duplicates())



         Fecha  Ruptura
0   2020-01-01        0
1   2020-01-02        0
2   2020-01-03        0
3   2020-01-04        0
4   2020-01-05        0
..         ...      ...
361 2020-12-27        0
362 2020-12-28        0
363 2020-12-29        0
364 2020-12-30        0
365 2020-12-31        0

[366 rows x 2 columns]


In [28]:
# Detectar outliers
outlier_thresholds = detect_outliers(all_processed_data)


Columna: Obrador Kiteni
 - Límite inferior: -6.2250000000000005
 - Límite superior: 10.375
 - Total de outliers: 48

Columna: PS#1
 - Límite inferior: -13.424999999999999
 - Límite superior: 22.375
 - Total de outliers: 46

Columna: KP-7+400
 - Límite inferior: -15.0
 - Límite superior: 25.0
 - Total de outliers: 46

Columna: KP 018+000
 - Límite inferior: -21.0
 - Límite superior: 35.0
 - Total de outliers: 35

Columna: KP 030+000
 - Límite inferior: -25.5
 - Límite superior: 42.5
 - Total de outliers: 24

Columna: KP 040+000
 - Límite inferior: -22.05
 - Límite superior: 36.75
 - Total de outliers: 31

Columna: KP 043+600
 - Límite inferior: -17.737499999999997
 - Límite superior: 29.562499999999996
 - Total de outliers: 36

Columna: KP 046+100
 - Límite inferior: -30.037499999999998
 - Límite superior: 50.0625
 - Total de outliers: 22

Columna: KP 050+000
 - Límite inferior: -31.612500000000004
 - Límite superior: 52.68750000000001
 - Total de outliers: 26

Columna: KP 057+000
 - Lí

# Exportar Resultados
Guarda los datos procesados en un archivo CSV consolidado.


In [29]:
all_processed_data.to_csv(OUTPUT_PATH, index=False, encoding='utf-8')
print(f"Datos consolidados guardados en: {OUTPUT_PATH}")


Datos consolidados guardados en: ../data/processed/2020/pluviometros-eda-2020.csv


# Graficos mensuales


In [30]:
import matplotlib.pyplot as plt
from matplotlib.backends.backend_pdf import PdfPages

# Crear un archivo PDF
OUTPUT_PATH = f"{PROCESSED_FOLDER}/{YEAR}/graphic-pluviometros-eda-{YEAR}.pdf"
with PdfPages(OUTPUT_PATH) as pdf:
    # Iterar sobre cada mes
    for mes in range(1, 13):
        # Filtrar los datos por mes
        data_mes = all_processed_data[pd.to_datetime(all_processed_data['Fecha']).dt.month == mes]
        promedios_mes = data_mes.mean(numeric_only=True)

        # Crear el gráfico
        plt.figure(figsize=(10, 6))
        promedios_mes.plot(kind='bar')
        plt.title(f"Promedio de Lluvia - Mes {mes}")
        plt.xlabel("Sensores/Tramos")
        plt.ylabel("Promedio de Lluvia")
        plt.xticks(rotation=45)
        plt.tight_layout()

        # Guardar el gráfico en el PDF
        pdf.savefig()  # Guarda la figura actual en el PDF
        plt.close()  # Cierra la figura para no sobrecargar la memoria

print(f"Gráficos mensuales guardados en {OUTPUT_PATH}")


Gráficos mensuales guardados en ../data/processed/2020/graphic-pluviometros-eda-2020.pdf
